In [10]:
import sys 
sys.path.append("..") 

import numpy as np
import matplotlib.pyplot as plt
import scipy as sp 

print("Numpy version: ", np.__version__)
print("Scipy version: ", sp.__version__)

from non_Hermitian_random_matrix import non_hermitian_symmetry_list, gap_types, is_complexification
from non_Hermitian_random_matrix import DeformHamiltonian, DeformHamiltonianGenerator

from machine_learning import Cluster
from topology_table import DeformTopologicalModel, topology_comparator
from functools import partial
import itertools

from multiprocessing import Pool
from tools import save_data, save_data_M, load_data, load_data_M

Numpy version:  1.23.5
Scipy version:  1.11.1


In [11]:
non_Hermitian_topological_classification_list_0D = {
    # Complex AZ class
    # real, imag, point
    "A":    ['Z', 'Z', '0'], 
    "AIII": ['0', 'Z+Z', 'Z'],
    # Real AZ class
    "AI":   ['Z', 'Z_2', 'Z_2'],
    "BDI":  ['Z_2', 'Z_2+Z_2', 'Z_2'],
    "D":    ['Z_2', 'Z_2', '0'],
    "DIII": ['0', 'Z', '2Z'],
    "AII":  ['2Z', '0', '0'], 
    "CII":  ['0', '0', '0'],
    "C":    ['0', '0', '0'],
    "CI":   ['0', 'Z', 'Z'],
    "AI+":  ['Z', 'Z', '0'],
    "BDI+": ['Z_2', 'Z+Z', 'Z'],
    # "D+":   ['D', 'AI', 'BDI'],
    "DIII+": ['0', 'Z', 'Z_2'],
    "AII+": ['2Z', '2Z', '0'],
    "CII+": ['0', '2Z+2Z', '2Z'],
    # "C+":   ['C', 'AII', 'CII'],
    "CI+":  ['0', 'Z', '0'],
    # Complex AZ class with sublattice symmetry
    "A:S":      ['0', '0', '0'], 
    "AIII:S+":  ['0', '0', '0'],
    "AIII:S-":  ['Z', 'Z', 'Z+Z'],
    # Real AZ class with sublattice symmetry
    "BDI:S++":  ['Z_2+Z_2', 'Z_2+Z_2', 'Z_2'],
    "DIII:S--":  ['0', '0', '0'],
    "CII:S++":  ['0', '0', '0'], 
    "CI:S--":   ['0', '0', '0'],
    "AI:S-":    ['0', '0', '0'],
    "BDI:S-+":  ['Z', 'Z_2', 'Z'],
    "D:S+":     ['Z_2', 'Z_2', '0'],
    "CII:S-+":  ['2Z', '0', 'Z'],
    "C:S+":     ['0', '0', '0'],
    "DIII:S++": ['0', '0', '0'],
    "CI:S++":   ['0', '0', 'Z_2'],
    "AI:S+":    ['Z_2', 'Z_2', 'Z_2+Z_2'],
    "BDI:S+-":  ['Z_2', 'Z_2', 'Z_2+Z_2'],
    "D:S-":     ['0', '0', '0'],
    "DIII:S+-": ['2Z', '2Z', '2Z+2Z'],
    "AII:S+":   ['0', '0', '0'],
    "CII:S+-":  ['0', '0', '0'],
    "C:S-":     ['0', '0', '0'],
    "CI:S+-":   ['Z', 'Z', 'Z+Z'],
}

In [ ]:
def worker(parameter, n_dim, n_sample):
    # print("Calculating {sym_class}: {gap_type} dim = {n_dim} n_band={n_band} n_sample={n_sample}\n".format(
    #     sym_class=non_Hermitian_symmetry_class, gap_type=gap_type, n_dim=n_dim, n_band=n_band, n_sample=n_sample))

    non_Hermitian_symmetry_class, gap_type, n_band = parameter
    
    index_gap_type = {'real line':0, 'imaginary line':1, 'point':2}
    class_type =  non_Hermitian_topological_classification_list_0D[non_Hermitian_symmetry_class][index_gap_type[gap_type]]
    n_sample_new = n_sample
    if class_type == 'Z+Z':
        n_sample_new =  np.max([n_sample, int(2*(n_band/2+1)**2)])
    elif class_type == "2Z+2Z":
        n_sample_new = np.max([n_sample, int(2*(n_band/4+1)**2)])
        
    
    filename = "./data_number_of_band/data_{symmetry_class}_{gap_type}_dim_{n_dim}_band_{n_band}.h5".format(
        symmetry_class=non_Hermitian_symmetry_class, gap_type=gap_type, n_dim=n_dim, n_band=n_band)
    
    flag = False
    try: 
        _x, _y = load_data(filename)
    except:
        flag = True

    if flag:
        generator  = DeformHamiltonianGenerator(
                n=n_band, n_dim=n_dim, 
                non_Hermitian_symmetry_class=non_Hermitian_symmetry_class, gap_type=gap_type,
                verbose=False)
        hamiltonians = generator.generate(n_sample=n_sample_new)
        models = [DeformTopologicalModel(hamiltonian=hamiltonian, gap_type=gap_type) for hamiltonian in hamiltonians]
        similarity_func = topology_comparator
        cluster = Cluster(similarity_function=similarity_func)
        center_indices, group_number = cluster.fit(models)
        
        filename = "./data_number_of_band/data_{symmetry_class}_{gap_type}_dim_{n_dim}_band_{n_band}.h5".format(
            symmetry_class=non_Hermitian_symmetry_class, gap_type=gap_type, n_dim=n_dim, n_band=n_band)
        save_data(center_indices, group_number, filename)
    
    #return len(group_number)

n_dim = 0
n_sample = 1000

n_bands = 8*np.array([1,2,3,6,8,10,12])
parameters = list(itertools.product(non_hermitian_symmetry_list, gap_types, n_bands))
print("Total case: ", len(parameters))

n_core = 32      
_worker = partial(worker, n_dim=n_dim, n_sample=n_sample)
with Pool(n_core) as pool:
    pool.map(_worker, parameters)

Total case:  798


In [ ]:
# parameters

# Run in parallel

In [ ]:
# n_sample = 1000

# def worker(non_Hermitian_symmetry_class, n_dim, n_band, gap_type, n_sample):
#     print("Calculating {sym_class}: {gap_type} dim = {n_dim} n_band={n_band} n_sample={n_sample}\n".format(
#         sym_class=non_Hermitian_symmetry_class, gap_type=gap_type, n_dim=n_dim, n_band=n_band, n_sample=n_sample))

#     generator  = DeformHamiltonianGenerator(
#             n=n_band, n_dim=n_dim, 
#             non_Hermitian_symmetry_class=non_Hermitian_symmetry_class, gap_type=gap_type,
#             verbose=False)
#     hamiltonians = generator.generate(n_sample=n_sample)
#     models = [DeformTopologicalModel(hamiltonian=hamiltonian, gap_type=gap_type) for hamiltonian in hamiltonians]
#     similarity_func = topology_comparator
#     cluster = Cluster(similarity_function=similarity_func)
#     center_indices, group_number = cluster.fit(models)
    
#     filename = "./data_number_of_band/data_{symmetry_class}_{gap_type}_dim_{n_dim}_band_{n_band}.h5".format(
#         symmetry_class=non_Hermitian_symmetry_class, gap_type=gap_type, n_dim=n_dim, n_band=n_band)
#     save_data(center_indices, group_number, filename)
    
#     #return len(group_number)

# n_core = 32
# for gap_type in ['imaginary line']:
#     n_dim = 0
#     # 8*np.array([1,2,3,6,8,10,12])
#     for n_band in 8*np.array([1,2,3,6]):
#         _worker = partial(worker, n_dim=n_dim, n_band=n_band, gap_type=gap_type, n_sample=n_sample)
#         with Pool(n_core) as pool:
#             pool.map(_worker, non_hermitian_symmetry_list)

In [ ]:
# n_sample = 1000

# def worker(non_Hermitian_symmetry_class, n_dim, n_band, gap_type, n_sample):
#     print("Calculating {sym_class}: {gap_type} dim = {n_dim} n_band={n_band} n_sample={n_sample}\n".format(
#         sym_class=non_Hermitian_symmetry_class, gap_type=gap_type, n_dim=n_dim, n_band=n_band, n_sample=n_sample))

#     generator  = DeformHamiltonianGenerator(
#             n=n_band, n_dim=n_dim, 
#             non_Hermitian_symmetry_class=non_Hermitian_symmetry_class, gap_type=gap_type,
#             verbose=False)
#     hamiltonians = generator.generate(n_sample=n_sample)
#     models = [DeformTopologicalModel(hamiltonian=hamiltonian, gap_type=gap_type) for hamiltonian in hamiltonians]
#     similarity_func = topology_comparator
#     cluster = Cluster(similarity_function=similarity_func)
#     center_indices, group_number = cluster.fit(models)
    
#     filename = "./data_number_of_band/data_{symmetry_class}_{gap_type}_dim_{n_dim}_band_{n_band}.h5".format(
#         symmetry_class=non_Hermitian_symmetry_class, gap_type=gap_type, n_dim=n_dim, n_band=n_band)
#     save_data(center_indices, group_number, filename)
    
#     #return len(group_number)

# n_core = 32
# for gap_type in ['point']:
#     n_dim = 0
#     # 8*np.array([1,2,3,6,8,10,12])
#     for n_band in 8*np.array([1,2,3,6,8,10,12]):
#         _worker = partial(worker, n_dim=n_dim, n_band=n_band, gap_type=gap_type, n_sample=n_sample)
#         with Pool(n_core) as pool:
#             pool.map(_worker, non_hermitian_symmetry_list)